In [7]:
!pip install openpyxl

You should consider upgrading via the 'C:\Users\lfdha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import numpy as np
import plotly.express as px
import pandas as pd
import pickle
import warnings
import openpyxl

In [3]:
def guardar_datos(datos, path):
    with open(path, "wb") as f:
        pickle.dump(datos,f)

def cargar_datos(path):
    with open(path, "rb") as f:
        return pickle.load(f)

In [4]:
path_df_with_topics = r'C:\Users\lfdha\Dropbox\Papers\Iberspeech_2022\Daily-Dialog-Zero-Shot.csv'
df_topics = pd.read_csv(path_df_with_topics)
df_topics = df_topics.loc[0:1000]
df_topics['topic'] = df_topics['topic'].str.capitalize()
df_topics['topic'] = df_topics['topic'].fillna('Other')
df_topics['topic'] = df_topics['topic'].str.replace('Nan', 'Other')
df_topics['topic'] = df_topics['topic'].str.replace('Teacher', 'Education/academic')
df_topics['topic'] = df_topics['topic'].str.replace('Climate conditions', 'Weather')
df_topics['topic'] = df_topics['topic'].str.replace('Newspaper', 'News')
df_topics.loc[df_topics['topic_score'] < 0.1, 'topic'] = 'Other'
df_topics['topic'] = df_topics['topic'].str.replace('Art', 'Other')
df_topics['topic'] = df_topics['topic'].str.replace('Politics', 'Other')
df_topics['topic'] = df_topics['topic'].str.replace('Videogames', 'Other')
df_topics['topic'] = df_topics['topic'].str.replace('Science', 'Other')
df_topics['topic'] = df_topics['topic'].str.replace('Education/academic', 'Other')
df_topics['topic'] = df_topics['topic'].str.replace('News', 'Other')
# df_topics['topic'] = df_topics['topic'].str.replace('Photography', 'Other')
# df_topics['topic'] = df_topics['topic'].str.replace('Weather', 'Other')
#df_topics.drop(df_topics[(df_topics['topic'] == 'Politics') | (df_topics['topic'] == 'Videogames')| (df_topics['topic'] == 'Science')].index, inplace=True)
df_topics.reset_index(inplace=True)


In [5]:
path_df_tagged_topics = r'C:\Users\lfdha\Dropbox\Papers\Iberspeech_2022\Daily-Dialog-Zero-Shot_for_labelling_ALL.xlsx'
df_tagged = pd.read_excel(path_df_tagged_topics, index_col=0)
df_tagged['topic'] = df_tagged['topic'].str.capitalize()
df_tagged['topic'] = df_tagged['topic'].fillna('Other')
#df_tagged.drop(df_tagged[(df_tagged['topic'] == 'Politics') | (df_tagged['topic'] == 'Videogames')| (df_tagged['topic'] == 'Science')].index, inplace=True)
#df_tagged.drop(df_tagged[(df_tagged['topic'] == 'Politics')].index, inplace=True)
df_tagged['topic'] = df_tagged['topic'].str.replace('Art', 'Other')
df_tagged['topic'] = df_tagged['topic'].str.replace('Politics', 'Other')
df_tagged['topic'] = df_tagged['topic'].str.replace('Videogames', 'Other')
df_tagged['topic'] = df_tagged['topic'].str.replace('Science', 'Other')
df_tagged['topic'] = df_tagged['topic'].str.replace('Education/academic', 'Other')
df_tagged['topic'] = df_tagged['topic'].str.replace('News', 'Other')
# df_tagged['topic'] = df_tagged['topic'].str.replace('Photography', 'Other')
# df_tagged['topic'] = df_tagged['topic'].str.replace('Weather', 'Other')

df_tagged.reset_index(inplace=True)


In [7]:
le = LabelEncoder().fit(df_topics['topic'])
possible_topics_mapping = dict()
possible_topics_mapping['topic_encoded'] = {l: i for i,l in enumerate(le.classes_)}
df_topics['topic_encoded'] = df_topics['topic']
df_tagged['topic_encoded'] = df_tagged['topic']

df_topics = df_topics.replace(possible_topics_mapping)
df_tagged = df_tagged.replace(possible_topics_mapping)

In [8]:
y_true = list(df_tagged['topic_encoded'].values)
y_pred = list(df_topics['topic_encoded'].values)

In [48]:
def get_most_likely_topic(data, threshold=0.6):
    list_topic_scores = ['best_first_level_score',
                        'best_second_level_score',
                        'best_third_level_score']

    max_df_idx = data[list_topic_scores].idxmax(axis=1)
    max_df_value = data[list_topic_scores].max(axis=1)
    topics = []
    for idx, best_topic in enumerate(max_df_idx):
        if 'first' in str(best_topic) and max_df_value[idx]>=threshold:
            topics.append(data['best_first_level_topic'].loc[idx])
            
        elif 'second' in str(best_topic) and max_df_value[idx]>=threshold:
            topics.append(data['best_second_level_topic'].loc[idx])
        elif 'third' in str(best_topic)and max_df_value[idx]>=threshold:
            if data['best_third_level_topic'].loc[idx] == 'Photography':
                topics.append('Other')
            else:
                topics.append(data['best_third_level_topic'].loc[idx])
        else:
            topics.append('Other')
    return topics

def get_topic_threshold(data, threshold=0.6):
    topics = [data['topic'].loc[idx] if data['topic_score'].loc[idx] > threshold else 'Other' for idx in range(data['topic'].shape[0]) ]
    return topics


def check_present_labels(y_true, y_pred, labels):
    present_labels = []
    present_labels.extend(y_true)
    present_labels.extend(y_pred)
    present_labels = list(set(present_labels))
    enc_labels = np.arange(0,len(labels))
    if len(present_labels) != len(labels):
        for i in enc_labels:
            if i not in present_labels:
                print(i)
                y_pred.append(i)
                y_true.append(0)
    return y_true, y_pred

def get_metrics(df_tagged, topics):
    df_topics = pd.DataFrame()
    df_topics['topic'] = topics
    df_topics['topic'] = df_topics['topic'].fillna('Other')
    df_topics['topic'] = df_topics['topic'].str.capitalize()
    df_topics['topic'] = df_topics['topic'].str.replace('media', 
                                                        'Media')
    df_topics['topic_encoded'] = df_topics['topic']
    df_topics = df_topics.replace(possible_topics_mapping)

    possible_topics = list()
    possible_topics.extend(df_topics['topic'].unique())
    possible_topics.extend(df_tagged['topic'].unique())
    possible_topics = list(set(possible_topics))

    labels = list(possible_topics_mapping['topic_encoded'].keys())

    y_true = list(df_tagged['topic_encoded'].values)
    y_pred = list(df_topics['topic_encoded'].values)

    y_true, y_pred = check_present_labels(y_true, y_pred, labels)

    f1scores = f1_score(y_true, y_pred, average=None)
    f1scores_weighted = f1_score(y_true, y_pred, average='weighted')
    f1scores_micro = f1_score(y_true, y_pred, average='micro')
    f1scores_macro = f1_score(y_true, y_pred, average='macro')
    precscores = precision_score(y_true, y_pred, average=None)
    recallscores = recall_score(y_true, y_pred, average=None)

    f1scores = f1_score(y_true, y_pred, average=None, zero_division=0)
    precscores = precision_score(y_true, y_pred, average=None, zero_division=0)
    recallscores = recall_score(y_true, y_pred, average=None, zero_division=0)
    print(classification_report(y_true, y_pred, target_names=labels))
    return labels, f1scores, precscores, recallscores,f1scores_weighted, f1scores_micro, f1scores_macro

def get_df_transponse(df):
    df_transposed = df.T
    df_transposed = df_transposed.rename(columns=df_transposed.iloc[0])
    df_transposed = df_transposed.drop(df_transposed.index[0])
    return df_transposed

def representation(df, labels, metric):
    import plotly.graph_objects as go
    import random
    df_ = df.copy()
    df_.reset_index(inplace=True)

    fig = go.Figure()
    colors = ['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen', 'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'grey', 'green', 'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', 'lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen', 'linen', 'magenta', 'maroon', 'mediumaquamarine', 'mediumblue', 'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 'mediumspringgreen', 'mediumturquoise', 'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy', 'oldlace', 'olive', 'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 'palevioletred', 'papayawhip', 'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'red', 'rosybrown', 'royalblue', 'saddlebrown', 'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 'slateblue', 'slategray', 'slategrey', 'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 'white', 'whitesmoke', 'yellow', 'yellowgreen']
    widths = [4]
    dashes = ['dash', 'dot', 'dashdot', 'solid']
    #thresholds = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    thresholds = np.arange(0.2, 1.0, 0.1).tolist()
    # Create and style traces
    for i, l in enumerate(df_.columns):
        if l == 'index':
            continue
        r = random.randint(0, len(colors))
        w = i % len(widths)
        d = i % len(dashes)
        #fig.add_trace(go.Scatter(x=thresholds, y=df_[l].tolist(), name=l, line=dict(color=colors[r], width=widths[w], dash=dashes[d])))
        fig.add_trace(go.Scatter(x=thresholds, y=df_[l].tolist(), mode = 'lines', name=l, line=dict(width=widths[w], dash=dashes[d])))

    # Edit the layout
    fig.update_layout(title=metric,
                    xaxis_title='ZS Threshold',
                    yaxis_title='Value', height=600)


    fig.update_layout(
        font=dict(
            #family="Courier New, monospace",
            size=22,  # Set the font size here
        )
    ) 

    #fig = px.line(df_, x="index", y=labels,
    #              labels={"index": "ZS Threshold"},
    #              title=metric, markers=True)
    fig.show()

In [44]:
warnings.filterwarnings('ignore') 
#thresholds = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
#thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
thresholds = np.arange(0.2, 1.0, 0.1).tolist()
first = True
col_cont=1
#df_topics.drop(df_topics[(df_topics['topic'] == 'Politics') | (df_topics['topic'] == 'Videogames')].index, inplace=True)
#df_topics.reset_index(inplace=True)

for threshold in thresholds:
    print(threshold)
    topics = get_topic_threshold(df_topics, threshold=threshold)
    labels,f1scores,precscores,recallscores,_,_,_ = get_metrics(df_tagged, topics)
    if first == True:
        # Create df
        f1_df = pd.DataFrame()
        prec_df = pd.DataFrame()
        recall_df = pd.DataFrame()
        f1_df['labels'] = labels
        prec_df['labels'] = labels
        recall_df['labels'] = labels
    first=False
    f1_df[str(threshold)] = f1scores
    prec_df[str(threshold)] = precscores
    recall_df[str(threshold)] = recallscores

f1_df_transposed = get_df_transponse(f1_df)
prec_df_transposed = get_df_transponse(prec_df)
recall_df_transposed = get_df_transponse(recall_df)

0.2
              precision    recall  f1-score   support

     Animals       0.43      0.83      0.57        12
       Books       0.29      0.50      0.36         4
        Cars       0.65      0.57      0.61        56
      Family       0.23      0.56      0.33        39
     Fashion       0.29      0.69      0.40       108
     Finance       0.60      0.63      0.61       119
        Food       0.87      0.79      0.83       184
      Movies       0.33      0.50      0.40         6
       Music       0.56      1.00      0.71        15
       Other       0.69      0.24      0.35       419
 Photography       0.11      0.25      0.15         4
      Sports       0.56      0.85      0.68        27
     Weather       0.12      0.71      0.21         7

    accuracy                           0.51      1000
   macro avg       0.44      0.63      0.48      1000
weighted avg       0.63      0.51      0.51      1000

0.30000000000000004
              precision    recall  f1-score   support



In [49]:
f1_columns = list(f1_df_transposed.columns.values)
#f1_columns.remove('Politics')
#f1_columns.remove('Videogames')
labels_aux = labels 
#labels_aux.remove('Politics')
#labels_aux.remove('Videogames')
f1_df_transposed_aux = f1_df_transposed[f1_columns]
representation(f1_df_transposed_aux, labels_aux, 'F1-Score')

In [30]:
topics = get_topic_threshold(df_topics, threshold=0.9)
labels,f1scores,precscores,recallscores, f1scores_weighted, f1scores_micro, f1scores_macro = get_metrics(df_tagged, topics)
def representation_specific_threshold(df, labels, metric, weighted=0.0, macro=0.0):
    df_ = df.copy()
    df_.reset_index(inplace=True)
    fig = px.bar(df_, x='labels', y='F1',
                  labels={"index": "Labels"},
                  title=metric)
    # fig.update_layout(uniformtext_minsize=28, uniformtext_mode='show')
    fig.add_hline(y=weighted, line_dash="dash", line_color="green", annotation_text='Weighted={:.3f}'.format(weighted))
    fig.add_hline(y=macro, line_dash="dash", line_color="red", annotation_text='Macro={:.3f}'.format(macro))
    # update_layout method used to modify change and size
    fig.update_layout(
        font=dict(
            #family="Courier New, monospace",
            size=24,  # Set the font size here
        )
    ) 
    fig.show()
df = pd.DataFrame()
df['labels'] = labels
df['F1'] = f1scores.tolist()
representation_specific_threshold(df, labels, 'F1', weighted=f1scores_weighted, macro=f1scores_macro)

              precision    recall  f1-score   support

     Animals       0.45      0.75      0.56        12
       Books       0.40      0.50      0.44         4
        Cars       0.83      0.36      0.50        56
      Family       0.70      0.41      0.52        39
     Fashion       0.64      0.36      0.46       108
     Finance       0.67      0.39      0.50       119
        Food       0.94      0.72      0.82       184
      Movies       1.00      0.50      0.67         6
       Music       0.65      1.00      0.79        15
       Other       0.62      0.88      0.73       419
 Photography       1.00      0.25      0.40         4
      Sports       0.73      0.59      0.65        27
     Weather       0.36      0.57      0.44         7

    accuracy                           0.67      1000
   macro avg       0.69      0.56      0.58      1000
weighted avg       0.70      0.67      0.66      1000

